<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [4]</a>'.</span>

## Step 1: Video & Ground Truth importieren

In [1]:
import pyVHR as vhr
import math
import numpy as np
import os
import glob
import configparser
from pprint import pprint
from pyVHR.BVP import *

config = configparser.ConfigParser(converters={'list': lambda x: [i.strip() for i in x.split(',')]})
config.read('config.ini')

vhr.plot.VisualizeParams.renderer = 'notebook'

data_folder = config['Paths']['data']
dataset_name = config['Pipe']['dataset']
video_DIR = data_folder + '/' + dataset_name
BVP_DIR = data_folder + '/' + dataset_name


dataset_hasGT = False


if (dataset_name == 'own'):
    dataset_hasGT = False 
    #TO-DO: Load own dataset
    dataset = {
        'numVideos': 0,
        'sigFilenames': [],
        'videoFilenames': [],
        'GT': []
    }
    
    #fh = open(data_folder)
    #pprint(fh.readline())
    for filename in glob.glob(data_folder + '\*.txt'):
        with open(os.path.join(os.getcwd(), filename), 'r') as f:
            text = f.read()
            print(text)
    
    for path, currentDirectory, files in os.walk(data_folder):
        for file in files:
            print(os.path.join(path, file))
            if file == 'GT.txt':
                text = open (os.path.join(path, file), "r") 
                dataset['GT'].append(float(text.read()))
                dataset['sigFilenames'].append(os.path.join(path, file))
                dataset['numVideos'] += 1
            if file == 'Vid.mp4':
                dataset['videoFilenames'].append(os.path.join(path, file))
else:
    dataset = vhr.datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)

print(dataset)
print(dataset_hasGT)

#pprint('Number of videos: ' + str(dataset.numVideos))
#pprint(vars(dataset))


C:/Users/lukas/unistuff/SS2022/ACEAI/project/data/own\JM\GT.txt
C:/Users/lukas/unistuff/SS2022/ACEAI/project/data/own\JM\Vid.mp4
C:/Users/lukas/unistuff/SS2022/ACEAI/project/data/own\MM1\GT.txt
C:/Users/lukas/unistuff/SS2022/ACEAI/project/data/own\MM1\Vid.mp4
C:/Users/lukas/unistuff/SS2022/ACEAI/project/data/own\MM2\GT.txt
C:/Users/lukas/unistuff/SS2022/ACEAI/project/data/own\MM2\Vid.mp4
C:/Users/lukas/unistuff/SS2022/ACEAI/project/data/own\MT1\GT.txt
C:/Users/lukas/unistuff/SS2022/ACEAI/project/data/own\MT1\Vid.mp4
C:/Users/lukas/unistuff/SS2022/ACEAI/project/data/own\MT2\GT.txt
C:/Users/lukas/unistuff/SS2022/ACEAI/project/data/own\MT2\Vid.mp4
C:/Users/lukas/unistuff/SS2022/ACEAI/project/data/own\RG1\GT.txt
C:/Users/lukas/unistuff/SS2022/ACEAI/project/data/own\RG1\Vid.mp4
C:/Users/lukas/unistuff/SS2022/ACEAI/project/data/own\RG2\GT.txt
C:/Users/lukas/unistuff/SS2022/ACEAI/project/data/own\RG2\Vid.mp4
{'numVideos': 7, 'sigFilenames': ['C:/Users/lukas/unistuff/SS2022/ACEAI/project/data/

In [2]:
wsize = 8 


#set signal extractor

sig_extractor = vhr.extraction.sig_processing.SignalProcessing()
#pprint(vars(sig_extractor))


if (config['Pipe']['skin_extractor'] == 'convex_hull'):
    skin_extractor = vhr.extraction.SkinExtractionConvexHull()
    

elif (config['Pipe']['skin_extractor'] == 'bisenet'):
    skin_extractor = vhr.extraction.SkinExtractionFaceParsing()

sig_extractor.set_skin_extractor(skin_extractor)

#loop through videos


errors = []

for x in range(dataset['numVideos']):
    video_idx = x
    videoFileName = dataset['videoFilenames'][video_idx]
    #print('Video processed name: ', videoFileName)

    fname = dataset['sigFilenames'][video_idx]
    fps = vhr.extraction.get_fps(videoFileName)
    #print('Video frame rate:     ',fps)
        
    if (config['Pipe']['skin_processor'] == 'holistic'):
        sig = sig_extractor.extract_holistic(videoFileName)
    
    elif (config['Pipe']['skin_processor'] == 'patch-based'):
        landmarks = vhr.extraction.MagicLandmarks.cheek_left_top +\
                       vhr.extraction.MagicLandmarks.forehead_center +\
                       vhr.extraction.MagicLandmarks.forehoead_right +\
                       vhr.extraction.MagicLandmarks.cheek_right_top +\
                       vhr.extraction.MagicLandmarks.forehead_left +\
                       vhr.extraction.MagicLandmarks.nose 

        sig_extractor.set_landmarks(landmarks)
        sig_extractor.set_square_patches_side(30.0)

        print('Num landmarks: ', len(landmarks))

        sig = sig_extractor.extract_patches(videoFileName, "squares", "mean")

    windowed_sig, timesES = vhr.extraction.sig_windowing(sig, wsize, 1, fps)
    
    #TO-DO: read from config
    holistic_used = True
    landmark_used = False
    
    print('Video ID: ' + str(video_idx))
    print('Num windows: ',len(windowed_sig))
    
    
    #filtering
    
    filters = config['Pipe'].getlist('filters')
    
    for filter in filters:
        if (filter == 'rgb'):
            rgb_low_thr = int(config.get('Pipe', 'rgb_low_thr'))
            rgb_high_thr = int(config.get('Pipe', 'rgb_high_thr'))
            rgb_params = {'RGB_LOW_TH': rgb_low_thr, 'RGB_HIGH_TH': rgb_high_thr}
            windowed_sig = vhr.BVP.apply_filter(windowed_sig, vhr.BVP.rgb_filter_th, params=rgb_params)
        elif (filter == 'bp'):
            bp_order = int(config.get('Pipe', 'bp_order'))
            bp_minHz = float(config.get('Pipe', 'bp_minHz'))
            bp_maxHz = float(config.get('Pipe', 'bp_maxHz'))
            bp_params = {'order': bp_order, 'minHz': bp_minHz, 'maxHz': bp_maxHz, 'fps':fps}
            windowed_sig = vhr.BVP.apply_filter(windowed_sig, vhr.BVP.BPfilter, params=bp_params)
        elif (filter == 'detrend'):
            windowed_sig = vhr.BVP.apply_filter(windowed_sig, vhr.BVP.detrend)
        elif (filter == 'sg-detrend'):
            windowed_sig = vhr.BVP.apply_filter(windowed_sig, vhr.BVP.sg_detrend)
        elif (filter == 'zscore'):
            windowed_sig = vhr.BVP.apply_filter(windowed_sig, vhr.BVP.zscore)
        elif (filter == 'zeromean'):
            windowed_sig = vhr.BVP.apply_filter(windowed_sig, vhr.BVP.zeromean)
    
    
    #bvp estimation
    if (config['Pipe']['method'] == 'cpu_POS' or config['Pipe']['method'] == 'cpu_SSR'):
        bvps = RGB_sig_to_BVP(windowed_sig, fps, device_type='cpu', method=globals()[config['Pipe']['method']], params={'fps':fps})
    elif (config['Pipe']['method'] == 'cpu_ICA' or config['Pipe']['method'] == 'cpu_PCA'):
        bvps = RGB_sig_to_BVP(windowed_sig, fps, device_type='cpu', method=globals()[config['Pipe']['method']], params={'component': 'second_comp'})
    else:
        bvps = RGB_sig_to_BVP(windowed_sig, fps, device_type='cpu', method=globals()[config['Pipe']['method']])

    
    #post filtering
    
#     post_filters = config['Pipe'].getlist('post_filters')
    
#     for post_filter in post_filters:
#         if (post_filter == 'rgb'):
#             rgb_low_thr = int(config.get('Pipe', 'rgb_low_thr'))
#             rgb_high_thr = int(config.get('Pipe', 'rgb_high_thr'))
#             rgb_params = {'RGB_LOW_TH': rgb_low_thr, 'RGB_HIGH_TH': rgb_high_thr}
#             bvps = vhr.BVP.apply_filter(bvps, vhr.BVP.rgb_filter_th, params=rgb_params)
#         elif (post_filter == 'bp'):
#             bp_order = int(config.get('Pipe', 'bp_order'))
#             bp_minHz = float(config.get('Pipe', 'bp_minHz'))
#             bp_maxHz = float(config.get('Pipe', 'bp_maxHz'))
#             bp_params = {'order': bp_order, 'minHz': bp_minHz, 'maxHz': bp_maxHz, 'fps':fps}
#             bvps = vhr.BVP.apply_filter(bvps, vhr.BVP.BPfilter, params=bp_params)
#         elif (post_filter == 'detrend'):
#             bvps = vhr.BVP.apply_filter(bvps, vhr.BVP.detrend)
#         elif (post_filter == 'sg-detrend'):
#             bvps = vhr.BVP.apply_filter(bvps, vhr.BVP.sg_detrend)
#         elif (post_filter == 'zscore'):
#             bvps = vhr.BVP.apply_filter(bvps, vhr.BVP.zscore)
#         elif (post_filter == 'zeromean'):
#             bvps = vhr.BVP.apply_filter(bvps, vhr.BVP.zeromean)

    
    #bvp to bpm
    
    bpmES = vhr.BPM.BVP_to_BPM(bvps, fps)
    median_bpmES, MAD = vhr.BPM.multi_est_BPM_median(bpmES)
    
    
    #visualization
    if (config['Pipe']['dataset'] != 'own'):
        if (config['Pipe']['skin_processor'] == 'patch-based'):
            vhr.plot.visualize_multi_est_BPM_vs_BPMs_list([median_bpmES, timesES], [[median_bpmES, timesES, "medianES"],[bpmGT, timesGT, "GT"]])
        elif (config['Pipe']['skin_processor'] == 'holistic'):
            vhr.plot.visualize_multi_est_BPM_vs_BPMs_list([bpmES, timesES], [[median_bpmES, timesES, "medianES"],[bpmGT, timesGT, "GT"]])

    if (config['Pipe']['skin_processor'] == 'patch-based'):
        bpmES = median_bpmES.tolist()


    
    error_GT_abs = [] # absolute value
    error_GT_rel = [] # relative value
    
    # erase differences of recorded time
    #min_list = max(timesGT[0], timesES[0])
    #print(min_list)
    
    #max_list = min(timesGT[len(timesGT)-1], timesES[len(timesES)-1])
    #print(max_list) 
    
      
    timesGT_erased = []
  
    #i = 0
    #while i < len(timesES):
     #   y = int(timesES[i])
      #  timesGT_erased.append(bpmGT[y])        
       # i=i+1
    
    #print(timesGT_erased)
    #print(len(timesGT_erased))
    
    zip_BPM = list(zip(bpmES, timesGT_erased))
    
    #print(zip_BPM)
       
    if dataset_hasGT:
        
        j = 0
        while j < len(zip_BPM):
            error_abs = zip_BPM[j][1] - zip_BPM[j][0]
            error_GT_abs.append(error_abs)

            error_rel = (zip_BPM[j][1] - zip_BPM[j][0])/zip_BPM[j][1]
            error_GT_rel.append(error_rel)
            j = j + 1


        #print(error_GT_abs)
        #print(error_GT_rel)

               
        avg_error_abs = math.fsum(error_GT_abs) / len(error_GT_abs)
        avg_error_rel = math.fsum(error_GT_rel)/len(error_GT_rel) * 100 

        print("Average absolute error: ", avg_error_abs, " degree")
        print("Average relative error: ", avg_error_rel, "%")
        print("_______________________________________________\n")

        #save result

        errors.append([avg_error_abs, avg_error_rel])    

    else:
        GT = dataset['GT'][video_idx]

        print(GT)
        print(len(bpmES))
        avg_bmpES = np.sum(bpmES) / len(bpmES)
        print(avg_bmpES)

        error_ownGT_abs = int(GT)-int(avg_bmpES)
        error_ownGT_rel = error_ownGT_abs*100/int(GT)
        print("Average absolute error: ", error_ownGT_abs, " degree")
        print("Average relative error: ", error_ownGT_rel, "%")
        errors.append([error_ownGT_abs, error_ownGT_rel])

sum = 0
deg = 0
for x in errors:
    deg = deg + x[0]
    sum = sum + x[1]
average = sum / len(errors)
averageDeg = deg / len(errors)
print('Average error over whole dataset: ', average, "%", 'or ', averageDeg, ' degree')
    
    

Video ID: 0
Num windows:  21
120.0
21
66.9050060453869
Average absolute error:  54  degree
Average relative error:  45.0 %


Video ID: 1
Num windows:  37
121.5
37
114.38151921452703
Average absolute error:  7  degree
Average relative error:  5.785123966942149 %


Video ID: 2
Num windows:  38
122.0
38
113.71522923519737
Average absolute error:  9  degree
Average relative error:  7.377049180327869 %


Video ID: 3
Num windows:  23
120.0
23
99.08491847826087
Average absolute error:  21  degree
Average relative error:  17.5 %


Video ID: 4
Num windows:  25
119.0
25
109.246611328125
Average absolute error:  10  degree
Average relative error:  8.403361344537815 %


Video ID: 5
Num windows:  25
117.5
25
84.118251953125
Average absolute error:  33  degree
Average relative error:  28.205128205128204 %


Video ID: 6
Num windows:  23
136.5
23
123.35311226222827
Average absolute error:  13  degree
Average relative error:  9.558823529411764 %
Average error over whole dataset:  17.404212318049684 % or  21.0  degree


In [3]:
results_file = config['Paths']['results']
with open(results_file, 'a') as f:
    f.write('Dataset: ')
    f.write(str(dataset_name))
    f.write('\n')
    f.write("Skin processing method: ")
    if (config['Pipe']['skin_processor'] == 'holistic'):
        f.write('Holistic\n')
    elif (config['Pipe']['skin_processor'] == 'patch-based'):
        f.write('Patch-based\n')
    f.write("Skin extractor: ")
    if (config['Pipe']['skin_extractor'] == 'convex_hull'):
        f.write('Convex Hull\n')
    elif (config['Pipe']['skin_extractor'] == 'bisenet'):
        f.write('BiSeNet\n')
        
    #TO-DO: Add Holistic / Patch-Based
    
    if len(filters) > 0:
        f.write('Filters:\n')
        
    if 'rgb' in filters:
        f.write('RGB: (Low Threshold ' + config['Pipe']['rgb_low_thr'] + ', ') 
        f.write('High Threshold ' + config['Pipe']['rgb_high_thr'] + ')\n')
        
    if 'bp' in filters:
        f.write('BP: (Order '+ config['Pipe']['bp_order'] + ', ')
        f.write('MinHz '+ config['Pipe']['bp_minHz'] + ', ')
        f.write('MaxHz '+ config['Pipe']['bp_maxHz'] + ')\n')
        
    if 'detrend' in filters:
        f.write('Detrend\n')
        
    if 'sg_detrend' in filters:
        f.write('SG-Detrend\n')
        
    if 'zscore' in filters:
        f.write('Z-Score\n')
        
    if 'zeromean' in filters:
        f.write('Zero Mean\n')
        
    f.write(config['Pipe']['method'] + '\n')

    f.write(str(average))
    f.write('%')
    f.write('\n')
    f.write(str(averageDeg))
    f.write(' Degree')
    f.write('\n')
    f.write('_____________________')
    f.write('\n')


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [4]:
methods = ['cpu_CHROM', 'cpu_LGI', 'cpu_POS', 'cpu_PBV', 'cpu_PCA', 'cpu_GREEN', 'cpu_ICA']
filters = ['rgb, bp, detrend', 'rgb, bp, sg_detrend', 'rgb, bp, zscore', 'rgb, bp, zeromean', '']
current = config['Pipe']['filters']
next_index = filters.index(current) + 1

current_str = 'filters = ' + current

if (next_index <= len(filters) - 1):
    with open('config.ini') as f:
        newText = f.read().replace(current_str, 'filters = ' + filters[next_index])

    with open('config.ini', "w") as f:
        f.write(newText)


ValueError: 'rgb, bp' is not in list